# Домашняя работа #2
## Пункт 1
По условию

$u(x, y) = \sin(\pi x)\sin(2\pi y)$

при определенном f(x, y), найдем его, подставив u в ДУ:

$f(x, y) = \sin(\pi x)\sin(2\pi y)(5 \pi^2 + e^{xy})$

## Пункт 2
Для численного решения этого уравнения может использоваться метод конечно-разностной дискретизации, аналогичный упражнению 2. Разделим интервал $[0, 1]$ по обеим координатам на $N+1$ отрезков одинаковой длины $h = 1/(N+1)$. В каждой точке $(x_i, y_j) = (ih, jh), i, j = 1,\ldots, N$ построим конечно-разностные аппроксимации вторых производных:
$$
-\frac{\partial^2}{\partial x^2}(x_i, yj) \approx -\frac{[u(x{i-1}, y_j) - 2u(x_i, yj) + u(x{i+1}, y_j)]}{h^2}
$$
$$
-\frac{\partial^2}{\partial y^2}(x_i, y_j) \approx -\frac{[u(xi, y{j-1}) - 2u(x_i, y_j) + u(xi, y{j+1})]}{h^2}
$$
Элементы СЛАУ
$$
A\cdot \overrightarrow u = \overrightarrow f,
$$
матрица и вектора-функции, записывается в терминах единого индекса $k = i + (j-1)N, k = 1, \ldots, N^2$. $(\overrightarrow u)_k = u(x_i, y_j), (\overrightarrow f)_k = f(x_i, yj)$.

Ненулевые элементы $A_{kk'}$ матрицы определяются следующим образом:
$$
A_{kk'} = 4/h^2 + e^{x_iyj}, i = i', j = j'
$$
$$
A_{kk'} = -1/h^2, |i - i'| = 1, j = j'
$$
$$
A_{kk'} = -1/h^2, i = i', |j - j'| = 1
$$

Напишем код, для генерации матрицы и вектора f.

In [ ]:
import numpy as np
from scipy import sparse
from scipy.sparse.linalg import inv
from timeit import default_timer as timer
import plotly.graph_objs as go
import plotly


def u_val(x, y):
    return np.sin(np.pi * x) * np.sin(2 * np.pi * y)


def f_val(x, y):
    return u_val(x, y) * (5 * np.pi ** 2 + np.exp(x * y))


def generate_F(x, y, func):
    res = []
    for yi in y:
        res.append(func(x, yi))
    return np.array(res)


def generate_A(x, y, h, n):
    res = np.zeros(shape=(n ** 2, n ** 2))
    for i in range(n):
        for j in range(n):
            k = i + j * n

            res[k][k] = 4 / h ** 2 + np.exp(x[i] * y[j])

            val = -1 / h ** 2
            # if |i - i'| = 1, j = j'
            if i + 1 < n:
                res[k][k + 1] = val
            if i - 1 >= 0:
                res[k][k - 1] = val

            # if |j - j'| = 1, i = i'
            if j + 1 < n:
                res[k][k + n] = val
            if j - 1 >= 0:
                res[k][k - n] = val
    return res

Для решения системы воспользуемся методом SOR.

In [ ]:
def SOR(A, f, w=1.0, max_iter=100):
    x = np.zeros_like(f)
    hist = []
    kk = []
    for k in range(max_iter):
        for i in range(len(x)):
            x[i] = (1 - w) * x[i] + w / A[i][i] * (f[i] - A[i, :i] @ x[:i] - A[i, i + 1:] @ x[i + 1:])
        hist.append(x.copy())
        kk.append(k + 1)
    return x, kk, hist

Построим графики зависимости логарифа нормы бесконечности разности ошибки на каждой итерации для разных $w$ при n = 10, 30 и 50. 

In [ ]:
plotly.offline.init_notebook_mode()

def gen_color(n):
    res = []
    for i in range(n):
        res.append({'color': f'rgba({255 / n * (i + 1)},'
                             f' 0,'
                             f' 0,'
                             f' 0.8)'
                    })
    return res


def plot(data):
    traces = []
    colors = gen_color(len(data))
    for (w, (x, y)), c in zip(data.items(), colors):
        traces.append(
            go.Scatter(
                x=x,
                y=y,
                mode='lines+markers',
                marker=c,
                name=str(w)
            )
        )

    layout = go.Layout(
        title='Зависимость точности от количества итераций',
        xaxis={
            'title': 'k'
        },
        yaxis={
            'title': 'Логарифм нормы бесконечности'
        }
    )

    fig = go.Figure(data=traces, layout=layout)
    plotly.offline.iplot(fig)

    
def test(n, w_arr=[1, 1.2, 1.4, 1.6, 1.8, 1.9]):
    h = 1 / (n + 1)
    ix = np.array([h * (i + 1) for i in range(n)])
    iy = np.array([h * (i + 1) for i in range(n)])

    f = generate_F(ix, iy, f_val).flatten()
    A = generate_A(ix, iy, h, n)

    u_true = generate_F(ix, iy, u_val).flatten()
    data = {}
    for w in w_arr:
        u_pred, x, y = SOR(A, f, w=w, max_iter=100)
        data[w] = (x, np.log10(np.linalg.norm(np.absolute(y - u_true), ord=np.inf, axis=1)))
        # print(np.linalg.norm(np.abs(u_true - u_pred), ord=np.inf))

    plot(data)


test(10)
test(30)
test(50)

Можем сделать вывод, что значение $w=1.9$ является оптимальным при возрастании N, на более маленьких значениях N, более малые значения $w$ показывают себя лучше, например 1.8. Теперь построим графики зависимости логарифа нормы бесконечности ошибки для разых N, при $w = 1.9$.

In [ ]:
def test2(w, n_arr):
    data = {}
    x = []
    y = []
    for n in n_arr:
        h = 1 / (n + 1)
        ix = np.array([h * (i + 1) for i in range(n)])
        iy = np.array([h * (i + 1) for i in range(n)])

        f = generate_F(ix, iy, f_val).flatten()
        A = generate_A(ix, iy, h, n)

        u_true = generate_F(ix, iy, u_val).flatten()

        u_pred, _, _ = SOR(A, f, w=w, max_iter=100)
        y.append(np.log10(np.linalg.norm(np.absolute(u_pred - u_true), ord=np.inf)))
        x.append(n)
    data[w] = (x, y)
    plot(data)

    
test2(w=1.9, n_arr=[10, 20, 30, 50, 70, 90])

После n=50 ошибка начинает возрастать, хотя казалось бы, что должно быть наоборот. Посмотрина итерации при n = 70:

In [ ]:
test(70, [1.9])

Получается, что алгоритму не хватает 100 итераций и следовало бы подумать о другом методе остановки алгоритма, а максимальное число итераций увеличить:

In [ ]:
def SOR(A, f, w=1.0, max_iter=1000, tol=1e-8):
    x = np.zeros_like(f)
    hist = []
    kk = []
    prev = x.copy()
    for k in range(max_iter):
        for i in range(len(x)):
            x[i] = (1 - w) * x[i] + w / A[i][i] * (f[i] - A[i, :i] @ x[:i] - A[i, i + 1:] @ x[i + 1:])
        hist.append(x.copy())
        kk.append(k + 1)

        if np.linalg.norm(x - prev) <= tol:
            break
        prev = x.copy()
    return x, kk, hist


def test(n, w_arr=(1, 1.2, 1.4, 1.6, 1.8, 1.9), tol=1e-3, max_iter=1000):
    h = 1 / (n + 1)
    ix = np.array([h * (i + 1) for i in range(n)])
    iy = np.array([h * (i + 1) for i in range(n)])

    f = generate_F(ix, iy, f_val).flatten()
    A = generate_A(ix, iy, h, n)

    u_true = generate_F(ix, iy, u_val).flatten()
    data = {}
    for w in w_arr:
        u_pred, x, y = SOR(A, f, w=w, max_iter=max_iter, tol=tol)
        data[w] = (x, np.log10(np.linalg.norm(np.absolute(y - u_true), ord=np.inf, axis=1)))
        # print(np.linalg.norm(np.abs(u_true - u_pred), ord=np.inf))
    plot(data)


def test2(w, n_arr, tol=1e-3, max_iter=1000):
    data = {}
    x = []
    y = []
    for n in n_arr:
        h = 1 / (n + 1)
        ix = np.array([h * (i + 1) for i in range(n)])
        iy = np.array([h * (i + 1) for i in range(n)])

        f = generate_F(ix, iy, f_val).flatten()
        A = generate_A(ix, iy, h, n)

        u_true = generate_F(ix, iy, u_val).flatten()

        u_pred, _, _ = SOR(A, f, w=w, tol=tol, max_iter=max_iter)
        y.append(np.log10(np.linalg.norm(np.absolute(u_pred - u_true), ord=np.inf)))
        x.append(n)
    data[w] = (x, y)
    plot(data)

    
test(70, [1.9], tol=1e-4)

Теперь метод сходится, как мы видим за 200 итераций. Понятно, что при большем n нам придется изменять tol и максимальное число итераций, но теперь мы можем контролировать сходимост и точность решения метода. Построим теперь теже самые графики.

In [ ]:
test2(w=1.9, n_arr=[10, 20, 30, 50, 70, 90], tol=1e-5)

Теперь график имеет ожидаемое поведение.